For the known pathogenic, codis, and medically relevant genes, ask
1) How many there are in the catalog
2) How many are in the benchmark
3) How many have >=5bp HG002 variants in the benchmark

In [ ]:
import pyranges

In [16]:
import pandas as pd

catalog = pd.read_csv("/Users/english/code/adotto/regions/adotto_TRregions_v1.2.bed", sep='\t')
catalog.set_index(['chr', 'start', 'end'], inplace=True)
benchmark = pd.read_csv("/Users/english/code/adotto/benchmark/GIABTR_benchmark.6.26/GIABTR.HG002.benchmark.regions.bed.gz",
                        sep='\t', names=['chr', 'start', 'end', 'tier', 'repl', 'var_state', 'entropy', 'mat_ad', 'pat_ad'])
benchmark.set_index(['chr', 'start', 'end'], inplace=True)

In [17]:
data = benchmark.join(catalog)

In [18]:
cat_in_cmg = pd.read_csv("catalog_hitting_mrg.bed", sep='\t',
                    names=['chr', 'start', 'end', 'gchr', 'gstart', 'gend', 'gene', 'ovl']).set_index(['chr', 'start', 'end'])

In [19]:
patho_meta = pd.read_csv("/Users/english/code/adotto/pathogenic/Patho.tsv", sep='\t')

In [20]:
ph_subset_c = catalog['patho'].isin(patho_meta[patho_meta['Repeat type'] == 'VNTR']['Locus'])
pheno = catalog[ph_subset_c]
bench_pheno = data[data['patho'].isin(patho_meta[patho_meta['Repeat type'] == 'VNTR']['Locus'])]

pa_subset = catalog['patho'].isin(patho_meta[patho_meta['Repeat type'] != 'VNTR']['Locus'])
patho = catalog[pa_subset]
bench_patho = data[data['patho'].isin(patho_meta[patho_meta['Repeat type'] != 'VNTR']['Locus'])]

codis = catalog[catalog['codis'] != '.']
bench_codis = data[data['codis'] != '.']

gene = pd.read_csv("GRCh38_mrg_full_gene.bed", sep='\t', names=['chr', 'start', 'end', 'gene'])
has_hit_cmg = cat_in_cmg[cat_in_cmg['gchr'] != '.']
bench_gene = data[data.index.isin(has_hit_cmg.index)]

In [21]:
def make_summary(d_bare, d_bench):
    """
    Okay, we gotta turn this into a table. Its too many numbers.
    Columns:
    - N : Total number of items there are (*on MRG as it'll have 1->N Gene to TRs
    - benchmark : Total number of items in benchmark
    - Tier1 count :
    - HG002 >=5bp Variant :
    - Non-HG002 >=5bp Variant :
    """
    ret = d_bench['tier'].value_counts()
    ret['N'] = len(d_bare)
    ret['benchmark'] = len(d_bench)
    ret['HG002 >=5bp'] = (d_bench['var_state'].apply(lambda x: (x & 0x1) != 0)).sum()
    ret['Other >=5bp'] = (d_bench['var_state'].apply(lambda x: (x & 0x4) != 0)).sum()
    return ret
    
    

In [22]:
p_summary = make_summary(patho, bench_patho)
p_summary.name = "Pathogenic"
c_summary = make_summary(codis, bench_codis)
c_summary.name = "CODIS"
g_summary = make_summary(has_hit_cmg, bench_gene)
g_summary.name = "MRG"
a_summary = make_summary(pheno, bench_pheno)
a_summary.name = "Phenotypic"


summary = pd.concat([p_summary, c_summary, g_summary, a_summary], axis=1).T

In [23]:
summary[["N", "benchmark", "Tier1", "Tier2", "HG002 >=5bp", "Other >=5bp"]]

,N,benchmark,Tier1,Tier2,HG002 >=5bp,Other >=5bp
Pathogenic,62,50,43,7,25,42
CODIS,51,44,26,18,25,44
MRG,299633,289964,278225,11739,17201,49219
Phenotypic,113,111,108,3,49,89


In [24]:
has_hit_cmg = cat_in_cmg[cat_in_cmg['gchr'] != '.']

In [25]:
summary['benchmark'] / summary['N']

Pathogenic    0.806452
CODIS         0.862745
MRG           0.967731
Phenotypic    0.982301
dtype: float64

In [26]:
summary['Tier1'] / summary['benchmark']

Pathogenic    0.860000
CODIS         0.590909
MRG           0.959516
Phenotypic    0.972973
dtype: float64

In [45]:
print("Patho:", (data['patho'] != '.').sum())
print("CODIS:", (data['codis'] != '.').sum())
print("CMG:", data.index.isin(has_hit_cmg.index).sum())

Patho: 50
CODIS: 44
CMG: 289964


In [46]:
print(50 / 62)
print(44 / 51)
print(289964 / 299633)

0.8064516129032258
0.8627450980392157
0.9677305236739612


In [21]:
data.columns

Index(['tier', 'repl', 'var_state', 'entropy', 'mat_ad', 'pat_ad', 'ovl_flag',
       'up_buff', 'dn_buff', 'hom_pct', 'n_filtered', 'n_annos',
       'n_subregions', 'mu_purity', 'pct_annotated', 'interspersed', 'patho',
       'codis', 'gene_flag', 'biotype', 'annos'],
      dtype='object')

In [22]:
print("Patho:", data[data['patho'] != '.']['tier'].value_counts())


Patho: Tier1    42
Tier2     8
Name: tier, dtype: int64


In [23]:
print("CODIS:", data[data['codis'] != '.']['tier'].value_counts())


CODIS: Tier1    26
Tier2    18
Name: tier, dtype: int64


In [47]:
print("CMG:", data[data.index.isin(has_hit_cmg.index)]['tier'].value_counts())

CMG: Tier1    278225
Tier2     11739
Name: tier, dtype: int64


In [48]:
278225 / (278225 + 11739)

0.9595156640134638

In [16]:
data[data['patho'] != '.']['var_state'].value_counts()

13    15
12    11
15     9
8      8
14     5
10     2
Name: var_state, dtype: int64

In [17]:
data[data['codis'] != '.']['var_state'].value_counts()

14    14
15    13
13    11
12     5
5      1
Name: var_state, dtype: int64

In [18]:
cat_view = data[data.index.isin(cat_in_cmg.index)]

In [19]:
cat_view['var_state'].value_counts()

8     115442
0      83822
10     40516
14     15774
12     13520
15      9720
13      6610
4       2569
5        754
2        159
11        26
9         17
6          7
1          4
7          1
Name: var_state, dtype: int64

Okay, we gotta turn this into a table. Its too many numbers.

Columns:
- Site : Pathogenic, CODIS, MRG
- N : Total number of items there are (*on MRG as it'll have 1->N Gene to TRs
- benchmark : Total number of items in benchmark
- Tier1 count :
- HG002 >=5bp Variant :
- Non-HG002 >=5bp Variant :

In [68]:
# How many mrg have variants
view = data[data.index.isin(has_hit_cmg.index)]

In [69]:
view['var_state'].value_counts()

8     115897
0      84002
10     40639
14     15856
12     13627
15      9751
13      6644
4       2576
5        757
2        159
11        26
9         18
6          7
1          4
7          1
Name: var_state, dtype: int64

In [71]:
view = data.join(has_hit_cmg)

In [91]:
v2 = view.groupby('gene')['var_state'].value_counts()
v2.name = 'count'
v2 = v2.reset_index()
want_states = v2['var_state'].apply(lambda x: x & 0x2 != 0)
result = v2[want_states].groupby('gene')['count'].sum()
print("Number of genes with >=5bp benchmark variant", len(result))
print("median", result.median())

Number of genes with >=5bp benchmark variant 4163


In [90]:
v2 = view[view['tier'] == 'Tier1'].groupby('gene')['var_state'].value_counts()
v2.name = 'count'
v2 = v2.reset_index()
want_states = v2['var_state'].apply(lambda x: x & 0x2 != 0)
result = v2[want_states].groupby('gene')['count'].sum()
print("Number of genes with >=5bp T1 benchmark variant", len(result))

Number of genes with >=5bp T1 benchmark variant 3956


In [93]:
result.median()

7.0

In [96]:
bench_patho[bench_patho['tier'] == 'Tier2']['repl'].value_counts()

__TN    8
Name: repl, dtype: int64

In [98]:
bench_codis[bench_codis['tier'] == 'Tier2']['repl'].value_counts()

__TN           17
TN_FN,FP_TN     1
Name: repl, dtype: int64

In [100]:
view[~view['gene'].isna() & (view['tier'] == "Tier2")]['repl'].value_counts()

__TN              11240
TN_TN_TP            104
FP_FP_TN             75
TP_TP_TN             53
TP_FP_TP             49
FP_TP_TP             49
FN_FN_TP             45
TP_TP_FN,FP          41
_FP_TN               26
TP_TP_FN             25
__TP                 22
FN,FP_FN,FP_TP       18
FP__TN               16
TP_TP_FP             16
TP_TN_TP             13
TN_TP_TP             11
FP_FP_TP             10
TN_TN_FP              8
FN_FN,FP_TP           8
FN,FP_FN_TP           8
FP_FN,FP_TP           6
TN_TN_FN,FP           6
TN_TP_TN              5
TN_TN_FN              5
TP_TN_TN              4
FN_FP_TP              4
_TN_TP                3
_FN_TP                3
_FP_TP                3
__FN                  3
FP__TP                2
_TP_FN                2
_FN,FP_TP             2
FP_FN_TP              2
FN,FP_FP_TP           2
_TP_TN                2
FN,FP_TN_TN           1
__FP                  1
TN_TN_UNK             1
FN,FP_TP_FP           1
FP_TP_FN              1
FN_FN_FP        

In [101]:
# Nevertheless, N% of these benchmark regions of interest containing HG002 ≥5bp variants remain Tier1


In [103]:
m_view = pd.concat([bench_codis, bench_patho, bench_gene])

In [106]:
print(len(m_view))

290058


In [109]:
m_view = m_view.reset_index().sort_values(by=['chr', "start", "end"]).drop_duplicates(subset=['chr', "start", "end"])
print(len(m_view))

290008


In [110]:
isHG002_5bpvariant = m_view['var_state'].apply(lambda x: (x & 0x2) != 0)
isTier1 = m_view['tier'] == "Tier1"

In [111]:
denom = isHG002_5bpvariant.sum()
neum = (isHG002_5bpvariant & isTier1).sum()
print(neum, denom, neum / denom)

64386 66465 0.9687203791469194
